# Mechanismy pozornosti a transformery

Jednou z hlavních nevýhod rekurentních sítí je, že všechna slova v sekvenci mají stejný vliv na výsledek. To vede k suboptimálnímu výkonu u standardních modelů LSTM encoder-decoder pro úlohy převodu sekvencí, jako je rozpoznávání pojmenovaných entit nebo strojový překlad. Ve skutečnosti mají konkrétní slova ve vstupní sekvenci často větší vliv na výstupy než jiná.

Zvažme model převodu sekvencí, například strojový překlad. Ten je implementován pomocí dvou rekurentních sítí, kde jedna síť (**encoder**) zkomprimuje vstupní sekvenci do skrytého stavu a druhá síť (**decoder**) tento skrytý stav rozvine do přeloženého výsledku. Problém tohoto přístupu spočívá v tom, že konečný stav sítě má potíže s uchováním informací z počátku věty, což vede k nízké kvalitě modelu u dlouhých vět.

**Mechanismy pozornosti** poskytují způsob, jak vážit kontextuální vliv jednotlivých vstupních vektorů na každou výstupní predikci RNN. To se implementuje vytvořením zkratek mezi mezistavy vstupní RNN a výstupní RNN. Tímto způsobem, při generování výstupního symbolu $y_t$, bereme v úvahu všechny skryté stavy vstupu $h_i$ s různými váhovými koeficienty $\alpha_{t,i}$.

![Obrázek zobrazující model encoder/decoder s aditivní vrstvou pozornosti](../../../../../translated_images/cs/encoder-decoder-attention.7a726296894fb567.webp)  
*Model encoder-decoder s mechanismem aditivní pozornosti podle [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citováno z [tohoto blogového příspěvku](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matice pozornosti $\{\alpha_{i,j}\}$ reprezentuje míru, do jaké určitá vstupní slova ovlivňují generování konkrétního slova ve výstupní sekvenci. Níže je příklad takové matice:

![Obrázek zobrazující příklad zarovnání nalezeného RNNsearch-50, převzato z Bahdanau - arviz.org](../../../../../translated_images/cs/bahdanau-fig3.09ba2d37f202a6af.webp)  

*Obrázek převzatý z [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Obr. 3)*

Mechanismy pozornosti jsou zodpovědné za velkou část současného nebo téměř současného stavu techniky v oblasti zpracování přirozeného jazyka. Přidání mechanismu pozornosti však výrazně zvyšuje počet parametrů modelu, což vedlo k problémům se škálováním u RNN. Klíčovým omezením škálování RNN je, že rekurentní povaha těchto modelů ztěžuje dávkování a paralelizaci tréninku. V RNN musí být každý prvek sekvence zpracován v sekvenčním pořadí, což znamená, že paralelizace není snadná.

Přijetí mechanismů pozornosti v kombinaci s tímto omezením vedlo k vytvoření dnes známých špičkových modelů Transformer, které používáme, od BERT po OpenGPT3.

## Modely Transformer

Namísto předávání kontextu každé předchozí predikce do dalšího kroku hodnocení používají **modely Transformer** **poziční kódování** a pozornost k zachycení kontextu daného vstupu v rámci poskytnutého textového okna. Obrázek níže ukazuje, jak poziční kódování s pozorností dokáže zachytit kontext v rámci daného okna.

![Animovaný GIF ukazující, jak jsou hodnocení prováděna v modelech Transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Protože každá vstupní pozice je mapována nezávisle na každou výstupní pozici, transformery umožňují lepší paralelizaci než RNN, což umožňuje vytvářet mnohem větší a expresivnější jazykové modely. Každá hlava pozornosti může být použita k učení různých vztahů mezi slovy, což zlepšuje úlohy zpracování přirozeného jazyka.

**BERT** (Bidirectional Encoder Representations from Transformers) je velmi velká vícevrstvá síť Transformer s 12 vrstvami pro *BERT-base* a 24 pro *BERT-large*. Model je nejprve předtrénován na velkém korpusu textových dat (WikiPedia + knihy) pomocí neřízeného učení (predikce maskovaných slov ve větě). Během předtrénování model absorbuje významnou úroveň porozumění jazyku, kterou lze následně využít s jinými datovými sadami pomocí doladění. Tento proces se nazývá **transfer learning**.

![Obrázek z http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/cs/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Existuje mnoho variant architektur Transformer, včetně BERT, DistilBERT, BigBird, OpenGPT3 a dalších, které lze doladit. Balíček [HuggingFace](https://github.com/huggingface/) poskytuje repozitář pro trénování mnoha těchto architektur pomocí PyTorch.

## Použití BERT pro klasifikaci textu

Podívejme se, jak můžeme použít předtrénovaný model BERT k řešení naší tradiční úlohy: klasifikace sekvencí. Budeme klasifikovat naši původní datovou sadu AG News.

Nejprve načtěme knihovnu HuggingFace a naši datovou sadu:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Protože budeme používat předtrénovaný model BERT, bude potřeba použít specifický tokenizer. Nejprve načteme tokenizer spojený s předtrénovaným modelem BERT.

Knihovna HuggingFace obsahuje repozitář předtrénovaných modelů, které můžete použít jednoduše tím, že zadáte jejich názvy jako argumenty funkcí `from_pretrained`. Všechny potřebné binární soubory pro model se automaticky stáhnou.

Nicméně, v určitých případech budete potřebovat načíst vlastní modely. V takovém případě můžete specifikovat adresář, který obsahuje všechny relevantní soubory, včetně parametrů pro tokenizer, souboru `config.json` s parametry modelu, binárních vah atd.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objekt `tokenizer` obsahuje funkci `encode`, kterou lze přímo použít k zakódování textu:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Poté vytvořme iterátory, které použijeme během tréninku k přístupu k datům. Protože BERT používá svou vlastní funkci kódování, budeme muset definovat funkci pro doplňování podobnou `padify`, kterou jsme definovali dříve:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

V našem případě budeme používat předtrénovaný model BERT nazvaný `bert-base-uncased`. Načtěme model pomocí balíčku `BertForSequenceClassfication`. To zajišťuje, že náš model již má požadovanou architekturu pro klasifikaci, včetně finálního klasifikátoru. Uvidíte varovnou zprávu, která uvádí, že váhy finálního klasifikátoru nejsou inicializovány a model by vyžadoval předtrénování - to je naprosto v pořádku, protože přesně to se chystáme udělat!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Teď jsme připraveni začít s tréninkem! Protože BERT je již předem natrénovaný, chceme začít s poměrně malou hodnotou učícího se kroku, abychom nezničili počáteční váhy.

Veškerou těžkou práci provádí model `BertForSequenceClassification`. Když model použijeme na tréninková data, vrátí jak ztrátu, tak výstup sítě pro vstupní minibatch. Ztrátu využíváme pro optimalizaci parametrů (`loss.backward()` provádí zpětný průchod) a `out` pro výpočet přesnosti tréninku porovnáním získaných štítků `labs` (vypočítaných pomocí `argmax`) s očekávanými `labels`.

Abychom měli proces pod kontrolou, akumulujeme ztrátu a přesnost během několika iterací a tiskneme je každých `report_freq` tréninkových cyklů.

Tento trénink pravděpodobně zabere poměrně dlouhou dobu, takže omezujeme počet iterací.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Můžete vidět (zejména pokud zvýšíte počet iterací a počkáte dostatečně dlouho), že klasifikace pomocí BERT nám poskytuje docela dobrou přesnost! To je proto, že BERT už velmi dobře rozumí struktuře jazyka, a my potřebujeme pouze doladit finální klasifikátor. Nicméně, protože BERT je velký model, celý proces trénování trvá dlouho a vyžaduje značnou výpočetní sílu! (GPU, a ideálně více než jedno).

> **Note:** V našem příkladu jsme používali jeden z nejmenších předtrénovaných modelů BERT. Existují větší modely, které pravděpodobně přinesou lepší výsledky.


## Hodnocení výkonu modelu

Nyní můžeme zhodnotit výkon našeho modelu na testovací sadě dat. Smyčka pro hodnocení je velmi podobná trénovací smyčce, ale nesmíme zapomenout přepnout model do hodnotícího režimu zavoláním `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Shrnutí

V této jednotce jsme si ukázali, jak snadné je vzít předtrénovaný jazykový model z knihovny **transformers** a přizpůsobit ho našemu úkolu klasifikace textu. Podobně lze modely BERT použít pro extrakci entit, odpovídání na otázky a další úlohy z oblasti NLP.

Modely transformerů představují současný špičkový stav v NLP a ve většině případů by měly být první volbou, se kterou začnete experimentovat při implementaci vlastních řešení NLP. Nicméně pochopení základních principů rekurentních neuronových sítí, o kterých jsme diskutovali v tomto modulu, je nesmírně důležité, pokud chcete vytvářet pokročilé neuronové modely.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
